In [ ]:
import sys
sys.path.append('/home/janvanrijn/projects/openml-python')
sys.path.append('/home/janvanrijn/projects/openml-python-contrib')
sys.path.append('/home/janvanrijn/projects/pymc3')

import arff
import openmlcontrib
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats
import seaborn as sns
import theano.tensor as tt

In [ ]:
x_columns = ['svc__gamma']
y_column = 'predictive_accuracy'

with open('/home/janvanrijn/projects/openml-multitask/data/svm-sklearn-bot.arff', 'r') as fp:
    dataset = arff.load(fp)
dataset = openmlcontrib.meta.arff_to_dataframe(dataset)
dataset = dataset[dataset['task_id'] == 6]
dataset = dataset[dataset['svc__kernel'] == 'rbf']

X = np.array(dataset[x_columns].values, dtype=np.float)
y = np.array(dataset[y_column].values, dtype=np.float)
train_size = 200


X_tr = X[:train_size]
X_te = X[train_size:]

y_tr = y[:train_size]
y_te = y[train_size:]

print(X_tr)

n_dims = X_tr.shape[1]

In [ ]:
# def warp_func(x, *args):
#     n_dims = args[0]
#     beta_dist = pm.distributions.continuous.Beta('beta_dist', args[1], args[2])
#     return beta_dist.logp(x)

with pm.Model() as marginal_gp_model:
    # Specify the covariance function.
    length_scales = []
    alpha_parameters = []
    beta_parameters = []
    for i in range(n_dims):
#         alpha = pm.distributions.continuous.Lognormal('a%d' % i, 0, 0.75)
#         alpha_parameters.append(alpha)
#         beta = pm.distributions.continuous.Lognormal('b%d' % i, 0, 0.75)
#         beta_parameters.append(beta)
#         length_scale = pm.distributions.continuous.Uniform('l%d' % i, 0, 10)
#         length_scales.append(length_scale)
        length_scales.append(1.0)
    
    cov_unwarped = pm.gp.cov.ExpQuad(n_dims, ls=length_scales)
    # cov = pm.gp.cov.WarpedInput(n_dims, warp_func=warp_func, args=tuple([n_dims] + alpha_parameters + beta_parameters), cov_func=cov_unwarped)

    # Specify the GP.  The default mean function is `Zero`.
    gp = pm.gp.Marginal(cov_func=cov_unwarped)

    # The scale of the white noise term can be provided,
    sigma = pm.HalfCauchy("sigma", beta=5)
    y_ = gp.marginal_likelihood("y", X=X_tr, y=y_tr, noise=sigma)
    trace = pm.sample(2000, progressbar=True)

In [ ]:
mu, var = gp.predict(X_te)

scipy.stats.spearmanr(mu, y_te)[0]